# Building a Spam Filter with Naive Bayes
## Introduction 
In this project, we're going to build a spam filter for SMS messages. Our goal is:
> to create a spam filter that classifies new messages with an accuracy greater than 80% .

So our first task is to "teach" the computer how to classify messages. To do that, we'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans. The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). Let's start by reading data set. 
## Overview

In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('SMSSpamCollection', sep = '\t', header=None, names=['Label', 'SMS'])

In [3]:
df.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df.shape

(5572, 2)

In [5]:
df['Label'].value_counts(normalize = True) *100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

The proportion of non-spam and spam messages is 87%/13% respectively. 

We're going to keep 80% of our dataset for training, and 20% for testing (we want to train the algorithm on as much data as possible, but we also want to have enough test data). The dataset has 5,572 messages, which means that:

- The training set will have 4,458 messages (about 80% of the dataset).
- The test set will have 1,114 messages (about 20% of the dataset).

## Sampling

In [6]:
# random state for repetative results
df_rand = df.sample(frac = 1, random_state=1)

In [7]:
sample_index = round(df.shape[0] * 0.8)
train = df_rand[:sample_index].reset_index(drop = True)
test = df_rand[sample_index:].reset_index(drop = True)
test

,Label,SMS
0,ham,Later i guess. I needa do mcat study too.
1,ham,But i haf enuff space got like 4 mb...
2,spam,Had your mobile 10 mths? Update to latest Oran...
3,ham,All sounds good. Fingers . Makes it difficult ...
4,ham,"All done, all handed in. Don't know if mega sh..."
...,...,...
1109,ham,"We're all getting worried over here, derek and..."
1110,ham,Oh oh... Den muz change plan liao... Go back h...
1111,ham,CERI U REBEL! SWEET DREAMZ ME LITTLE BUDDY!! C...
1112,spam,Text & meet someone sexy today. U can find a d...


In [8]:
# checking the proportion of traing model
train['Label'].value_counts(normalize = True)*100

ham     86.54105
spam    13.45895
Name: Label, dtype: float64

To classify a new message, we will use [Naive Bayes algorithm](https://en.wikipedia.org/wiki/Naive_Bayes_classifier). Especially, we will compare the probability of being spam and non-spam. To calculate all these probabilities, we'll first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need.

## Cleaning Data Set

In [9]:
# removing non-word characters, making them in lower case and split into different words
train['SMS'] = train['SMS'].str.replace('\W', ' ').str.lower().str.split()

## Creating the Vocabulary

In [10]:
# all unique words
fq = []
def get_col(row):
    for i in row:
        if i not in fq:
            fq.append(i)
train['SMS'].apply(get_col);

In [11]:
#expand dict with zeros
unique_voc = {unique_word: [0] * len(fq) for unique_word in fq}

In [12]:
# we'd like to create a table with words as columns and values as the amount of word appearance for every sentence
for index, sms in enumerate(train['SMS']):
    for word in sms:
        unique_voc[word][index] += 1

uv_df = pd.DataFrame(unique_voc)
new_train = pd.concat([train, uv_df], axis = 1)

In [13]:
new_train.head()

,Label,SMS,yep,by,the,pretty,sculpture,yes,princess,are,...,beauty,hides,secrets,n8,jewelry,related,trade,arul,bx526,wherre
0,ham,"[yep, by, the, pretty, sculpture]",1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Calculating Probability
### Constants
To calculate P(wi|Spam) and P(wi|Ham), we will use these equations:

![Image](https://render.githubusercontent.com/render/math?math=P%28w_i%7CHam%29%20%3D%20%5Cfrac%7BN_%7Bw_i%7CHam%7D%20%2B%20%5Calpha%7D%7BN_%7BHam%7D%20%2B%20%5Calpha%20%5Ccdot%20N_%7BVocabulary%7D%7D&mode=display)

Also, we will use Laplace smoothing and set a = 1

In [20]:
# Separate tables
df_spam = new_train[new_train['Label'] == 'spam']
df_ham = new_train[new_train['Label'] == 'ham']

In [15]:
# variables to calculate the probabilty

# P_spam and P_ham
p_spam = df_spam.shape[0]/new_train.shape[0]
p_ham = df_ham.shape[0]/new_train.shape[0]

# N_spam and N_ham
n_spam = df_spam['SMS'].apply(len).sum()
n_ham = df_ham['SMS'].apply(len).sum()

# N vocabulary
n_vocabulary = len(fq)

#Laplace smoothing
a = 1

### Spam and Ham probabilities

In [16]:
spam_dict = {}
for i in df_spam.columns[2:]:
    spam_dict[i] = (df_spam[i].sum() + 1)/(n_spam + n_vocabulary)
    
ham_dict = {}
for i in df_ham.columns[2:]:
    ham_dict[i] = (df_ham[i].sum() + 1)/(n_ham + n_vocabulary)

## Classifying a new message
The function bellow will take a new message, calculate the probabilities of being spam and non-spam, compare these values and:
1. If P(Spam) > P(Ham) - mark as spam
2. If P(Ham) > P(Spam) - mark as non-spam
3. If P(Ham) == P(Spam) - algorith request human help

In [17]:
def classify(message):
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for i in message:
        if i in spam_dict:
            p_spam_given_message *= spam_dict[i]
            
        if i in ham_dict:
            p_ham_given_message *= ham_dict[i]
            
    if (p_spam_given_message > p_ham_given_message):
        return 'spam'
    elif (p_spam_given_message < p_ham_given_message):
        return 'ham'
    else:
        return 'needs human classification'

### Measuring the spam filter's accuracy

In [23]:
test['predicted'] = test['SMS'].apply(classify)
print('The percent of wrong predictions is {:.2%}'.format((test['Label'] != test['predicted']).sum()/test.shape[0]))

The percent of wrong predictions is 1.26%


In [21]:
test.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


The final accuracy is 98.74% which is really good.